# Mortgage file to SQL database

load necessary packages

In [1]:
import pandas as pd
from datetime import datetime
import pickle
import sqlite3
import dask.dataframe as dd
from dask.diagnostics.progress import ProgressBar
from tqdm import tqdm
import numpy as np

## load necessary dataset

In [31]:
# load file_layout
layout = pd.read_excel("../../Data/mortgage_data/file_layout.xlsx", sheet_name=None)

In [32]:
# # Extract column names and data types for both origination and performance datasets
# orig_layout = layout['Origination Data File']
# perf_layout = layout['Monthly Performance Data File']

# # Extract column names and data types
# orig_column_names = orig_layout['ATTRIBUTE NAME'].tolist()
# perf_column_names = perf_layout['ATTRIBUTE NAME'].tolist()

# orig_column_force_types = orig_layout['DATA TYPE'].tolist()

# #create dict to force data types only if force type is not None
# orig_column_force_types = dict(zip(orig_column_names, orig_column_force_types))
# orig_column_force_types = {k: v for k, v in orig_column_force_types.items() if v != 0}

# cols_keep_perf = perf_layout['KEEP'].tolist()
# cols_keep_orig = orig_layout['KEEP'].tolist()



### Load the origination and the performance datasets into dictionary. Also drop unnecessary columns.

In [35]:
def load_yearly_data(year, base_dir="../../Data/mortgage_data", layout = layout):
    """
    Load and format the origination and performance datasets for a given year, considering the folder structure.
    
    Parameters:
    - year: The year for which to load the data.
    - base_dir: The base directory where the datasets are stored.
    
    Returns:
    - orig_data: Formatted origination dataset for the given year.
    - perf_data: Formatted performance dataset for the given year.
    """
    # Extract column names and data types for both origination and performance datasets
    orig_layout = layout['Origination Data File']
    perf_layout = layout['Monthly Performance Data File']

    # Extract column names and data types
    orig_column_names = orig_layout['ATTRIBUTE NAME'].tolist()
    perf_column_names = perf_layout['ATTRIBUTE NAME'].tolist()

    orig_column_force_types = orig_layout['DATA TYPE'].tolist()
    
    #create dict to force data types only if force type is not None
    orig_column_force_types = dict(zip(orig_column_names, orig_column_force_types))
    orig_column_force_types = {k: v for k, v in orig_column_force_types.items() if v != 0}
    # display(orig_column_force_types)
    cols_keep_perf = perf_layout['KEEP'].tolist()
    cols_keep_orig = orig_layout['KEEP'].tolist()
    del orig_layout, perf_layout, layout
    
    # Construct file paths considering the "sample_YYYY" folder structure
    orig_file_path = f"{base_dir}/sample_{year}/sample_orig_{year}.txt"
    perf_file_path = f"{base_dir}/sample_{year}/sample_svcg_{year}.txt"
    

    # Load origination data, use orig_column_force_types to force data types
    orig_data = pd.read_csv(orig_file_path, sep="|", header=None, low_memory=False, names = orig_column_names, dtype= orig_column_force_types)
    #select only the first 22 columns
    orig_data = orig_data.iloc[:, 0:22]
    # Load performance data
    perf_data = pd.read_csv(perf_file_path, sep="|", header=None, names=perf_column_names, low_memory=False)
        #function that drops columns where cols_keep is 0
    def drop_cols(data, cols_keep, col_names):
        cols_to_drop = [col_names[i] for i, val in enumerate(cols_keep) if val == 0]
        return data.drop(columns=cols_to_drop)
    try:
        orig_data = drop_cols(orig_data, cols_keep_orig[0:22], orig_column_names)
        perf_data = drop_cols(perf_data, cols_keep_perf, perf_column_names)
        # display('cols dropped')
    except:
        # display('no cols dropped')
        pass
    orig_data = orig_data.dropna(axis=1, how='all')
    perf_data = perf_data.dropna(axis=1, how='all')
    return orig_data, perf_data

def load_all_datasets(start_year=1999, end_year=2022, base_dir="../../Data/mortgage_data/"):
    """
    Load all origination and performance datasets for a given range of years.
    
    Parameters:
    - start_year: The starting year (inclusive) for which to load the data.
    - end_year: The ending year (inclusive) for which to load the data.
    - base_dir: The base directory where the datasets are stored.
    
    Returns:
    - datasets: Dictionary containing formatted origination and performance datasets for the given range of years.
    """
    
    datasets = {}
    
    for year in tqdm(range(start_year, end_year + 1), desc = "Loading datasets"):
        orig_data, perf_data = load_yearly_data(year, base_dir=base_dir)
        datasets[f"orig_{year}"] = orig_data
        datasets[f"perf_{year}"] = perf_data
    return datasets

datasets_tot = load_all_datasets(start_year=1999, end_year=2022)
# del cols_keep_orig, cols_keep_perf, orig_column_names, perf_column_names
#Takes 1:30 to run

Loading datasets:   0%|          | 0/24 [00:00<?, ?it/s]

dtype('O')

Loading datasets:   4%|▍         | 1/24 [00:04<01:33,  4.05s/it]

dtype('O')

Loading datasets:   8%|▊         | 2/24 [00:06<01:03,  2.87s/it]

...

Loading datasets:  92%|█████████▏| 22/24 [01:34<00:05,  2.80s/it]

dtype('O')

Loading datasets:  96%|█████████▌| 23/24 [01:35<00:02,  2.38s/it]

dtype('O')

Loading datasets: 100%|██████████| 24/24 [01:36<00:00,  4.01s/it]


### Merge Origination Dataset with Performance Dataset on LSN

In [39]:
def merge_all_datasets(datasets):
    """
    Merge all origination and performance datasets within the provided dictionary according to their year.
    
    Parameters:
    - datasets: Dictionary containing formatted origination and performance datasets.
    
    Returns:
    - merged_datasets: Dictionary containing merged datasets for each year.
    """
    
    def merge_orig_with_perf(orig_data, perf_data):
        merged_data = pd.merge(perf_data, orig_data, on="LSN", how="left")
        return merged_data
    
    merged_datasets = {}
    # Extract the range of years from the dataset keys
    years = sorted(set(int(key.split("_")[-1]) for key in datasets.keys()))
    for year in tqdm(years, desc="Merging datasets"):
        orig_key = f"orig_{year}"
        perf_key = f"perf_{year}"
        if orig_key in datasets and perf_key in datasets:
            merged_data = merge_orig_with_perf(datasets[orig_key], datasets[perf_key])
            merged_data['Date'] = pd.to_datetime(merged_data['MRP'], format = '%Y%m')
            merged_data = merged_data.drop(['MRP'], axis=1)
            merged_data = merged_data[["LSN"] + ["Date"] + [col for col in merged_data.columns if col != "LSN" and col != "Date"]]
            merged_datasets[f"fm_{year}"] = merged_data
            # print("merged", year)
    return merged_datasets

# Merge all datasets in the provided dictionary (in this case, datasets_demo)
merged_datasets = merge_all_datasets(datasets_tot)
merged_datasets.keys()  # Display the keys of the merged datasets dictionary
del datasets_tot

Merging datasets: 100%|██████████| 24/24 [00:23<00:00,  1.04it/s]


In [40]:
merged_datasets['fm_2006'].dtypes

Date       datetime64[ns]
LSN                object
CLDS               object
AGE                 int64
CIR               float64
ELTV              float64
DDD                object
CS                  int64
FPD                 int64
FIRST_F            object
MD                  int64
MSA                object
MIP                 int64
CLTV                int64
DTI                 int64
LTV                 int64
OIR               float64
P_TYPE             object
POSTAL             object
OLT                 int64
dtype: object

In [41]:
def process_dataset(df):
    # Transformations
    # df['3ZIP'] = df['POSTAL'].astype(str).str[:3].astype(str)
    # df['STATE'] = df['POSTAL'].apply(postal_to_state)
    df['DDD'] = df['DDD'].fillna(0).replace('Y', 1)
    df['FIRST_F'] = df['FIRST_F'].replace({'N': 0, 'Y': 1})
    #change RA values to 99 in CLDS
    df['CLDS'] = df['CLDS'].replace('RA', 99)
    df['CLDS'] = df['CLDS'].astype('int16')
    # df['D90'] = 0
    # df['D180'] = 0
    # # Process group
    # def process_group(group):
    #     for val, offset, column in [(3, 0, 'D90'), (7, 0, 'D180')]:
    #         if val in group['CLDS'].values:
    #             delinquency_date = group[group['CLDS'] == val]['Date'].min()
    #             back_date = delinquency_date - pd.DateOffset(months=offset)
    #             group.loc[group['Date'] == back_date, column] = 1
    #     return group
    
    # df = df.groupby('LSN').apply(process_group, meta=df._meta)
    # #ungroup df
    # df = df.reset_index(drop=True)
    # Move Date and 3ZIP to the front
    df = df[["Date", "MSA"] + [col for col in df.columns if col not in ["Date", "MSA"]]]
    
    return df

# Convert to Dask DataFrames and process
with ProgressBar():
    for key in merged_datasets.keys():
        print(f"{key} processing...")
        ddf = dd.from_pandas(merged_datasets[key], npartitions=6)
        merged_datasets[key] = process_dataset(ddf).compute()

fm_1999 processing...
[########################################] | 100% Completed | 1.11 ss
fm_2000 processing...
[########################################] | 100% Completed | 615.05 ms
fm_2001 processing...
[########################################] | 100% Completed | 981.12 ms
fm_2002 processing...
[########################################] | 100% Completed | 1.27 ss
fm_2003 processing...
[########################################] | 100% Completed | 1.93 ss
fm_2004 processing...
[########################################] | 100% Completed | 1.88 ss
fm_2005 processing...
[########################################] | 100% Completed | 1.88 ss
fm_2006 processing...
[########################################] | 100% Completed | 1.47 ss
fm_2007 processing...
[########################################] | 100% Completed | 1.38 ss
fm_2008 processing...
[########################################] | 100% Completed | 1.10 sms
fm_2009 processing...
[########################################] | 100% Com

In [42]:
#save merged datasets to pickle one by one
for key in merged_datasets.keys():
    merged_datasets[key].to_pickle(f"../../Data/mortgage_data/processed/{key}.pkl")
    print(f"{key} saved")

fm_1999 saved
fm_2000 saved
fm_2001 saved
fm_2002 saved
fm_2003 saved
fm_2004 saved
fm_2005 saved
fm_2006 saved
fm_2007 saved
fm_2008 saved
fm_2009 saved
fm_2010 saved
fm_2011 saved
fm_2012 saved
fm_2013 saved
fm_2014 saved
fm_2015 saved
fm_2016 saved
fm_2017 saved
fm_2018 saved
fm_2019 saved
fm_2020 saved
fm_2021 saved
fm_2022 saved


## Open connection SQL

In [43]:
db_path = "../../Database/thesis_database.db"
conn = sqlite3.connect(db_path)

Store mortgage dataset into SQL database

In [44]:
for key, dataset in merged_datasets.items():
    print("Writing", key, "to database...")
    dataset.to_sql(key, conn, if_exists = "replace", index = False)

Writing fm_1999 to database...
Writing fm_2000 to database...
Writing fm_2001 to database...
Writing fm_2002 to database...
Writing fm_2003 to database...
Writing fm_2004 to database...
Writing fm_2005 to database...
Writing fm_2006 to database...
Writing fm_2007 to database...
Writing fm_2008 to database...
Writing fm_2009 to database...
Writing fm_2010 to database...
Writing fm_2011 to database...
Writing fm_2012 to database...
Writing fm_2013 to database...
Writing fm_2014 to database...
Writing fm_2015 to database...
Writing fm_2016 to database...
Writing fm_2017 to database...
Writing fm_2018 to database...
Writing fm_2019 to database...
Writing fm_2020 to database...
Writing fm_2021 to database...
Writing fm_2022 to database...


In [97]:
#close connection
conn.close()

### Connection Closed SQL

# TEST

In [2]:
db_path = "../../Database/thesis_database.db"
conn = sqlite3.connect(db_path)

In [3]:
query   =   """
SELECT *
FROM fm_2005
"""
fm_2005 = pd.read_sql(query, conn)

In [4]:
#show number of unique loans
print(fm_2005['LSN'].nunique())
#show number of loans that do not have MSA (NaN)
print(fm_2005[fm_2005['MSA'].isna()]['LSN'].nunique())

#show percentage of loans which does not have MSA
print(fm_2005[fm_2005['MSA'].isna()]['LSN'].nunique()/fm_2005['LSN'].nunique())
fm_2005_msa = fm_2005[fm_2005['MSA'].notna()]

49994
8007
0.16015921910629274


In [5]:
#load mainlang_usa_gdf_msa.pkl
with open('../../Data/mainland_usa_gdf_msa.pkl', 'rb') as f:
    mainland_usa_gdf_msa = pickle.load(f)


In [6]:
print(mainland_usa_gdf_msa['MSA'].duplicated().sum())

741


In [7]:
# Step 1: Group by 'MSA' and aggregate
def custom_agg(series):
    if np.issubdtype(series.dtype, np.number):
        return series.mean()
    else:
        return series.iloc[0]

agg_funcs = {col: custom_agg for col in mainland_usa_gdf_msa.columns if col != 'MSA'}

cols_to_drop = ['HRCN_HLRR', 'HRCN_RISKR', 'HRCN_EALR', 'HRCN_RISK_CATEGORY_QUANTILE', 'HRCN_EALS_Norm']
mainland_usa_gdf_msa = mainland_usa_gdf_msa.drop(columns=cols_to_drop)

mainland_usa_gdf_msa_aggregated = mainland_usa_gdf_msa.groupby('MSA').mean(agg_funcs).reset_index()
# Step 2: Merge the DataFrames
fm_2005_msa_hrcn_2 = fm_2005_msa.merge(mainland_usa_gdf_msa_aggregated, on='MSA', how='inner')


In [53]:
#Save dictionary to accessible pickle file
with open("../../Data/pickle/mortgage_data/fm_datasets.pickle", "wb") as f:
    pickle.dump(merged_datasets, f)

FileNotFoundError: [Errno 2] No such file or directory: '../../Data/pickle/mortgage_data/fm_datasets.pickle'

## Open connection SQL

In [10]:
db_path = "../../Database/thesis_database.db"
conn = sqlite3.connect(db_path)

Store mortgage dataset into SQL database

In [11]:
for key, dataset in merged_datasets.items():
    print("Writing", key, "to database...")
    dataset.to_sql(key, conn, if_exists = "replace", index = False)

Writing fm_1999 to database...
Writing fm_2000 to database...
Writing fm_2001 to database...
Writing fm_2002 to database...
Writing fm_2003 to database...
Writing fm_2004 to database...
Writing fm_2005 to database...
Writing fm_2006 to database...
Writing fm_2007 to database...
Writing fm_2008 to database...
Writing fm_2009 to database...
Writing fm_2010 to database...
Writing fm_2011 to database...
Writing fm_2012 to database...
Writing fm_2013 to database...
Writing fm_2014 to database...
Writing fm_2015 to database...
Writing fm_2016 to database...
Writing fm_2017 to database...
Writing fm_2018 to database...
Writing fm_2019 to database...
Writing fm_2020 to database...
Writing fm_2021 to database...
Writing fm_2022 to database...


In [12]:
#close connection
conn.close()

## Connection Closed SQL

In [34]:
#load fm_2007
fm_2007 = pd.read_pickle("../../Data/mortgage_data/processed/fm_2007.pkl")
#Count MSA NaN
print(fm_2007['MSA'].isna().sum())
#Percentage of MSA NaN
print(fm_2007['MSA'].isna().sum()/len(fm_2007))
#Remove MSA NaN
fm_2007 = fm_2007.dropna(subset=['MSA'])

#extract MSA and POSTAL from fm_2007
msa_2007 = fm_2007[['MSA', 'POSTAL']]
msa_2007['MSA'] = msa_2007['MSA'].astype(str)

#Output true if there are MSA of length 4
msa_2007['MSA_4'] = msa_2007['MSA'].str.len() == 5
#print number of MSA of length 4
print(msa_2007['MSA_4'].sum())

618876
0.2105044752218223
0
